In [2]:
import pymongo
from datetime import date, datetime
import math
import random
from bson.objectid import ObjectId

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["dskola"]
mycol = mydb["books"]

for i in range(0, 100) :

    my_dict = {
        "title": "Book Title " + str(i),
        "author": "Author " + str((i % 10 + 1)), # 10 different authors
        "genre": random.choice(["Fiction", "Dystopian", "Adventure", "Science Fiction", "Fantasy"]), # 5 different genres
        "published_year": 1950 + (i % 70), # Published between 1950 and 2019
        "copies_available": math.floor(random.randint(1, 9) * 10) + 1, # 1 to 10 copies available
        "borrowed_by": [
            {
                "user_id": ObjectId(),
                "borrowed_date": datetime(2024, random.randint(1, 12), random.randint(1, 30)), # Borrowed dates spread over the year 2024
                "return_date": None
            }
        ]
    }
    mycol.insert_one(my_dict)


In [3]:
# Find all books authored by "Author 5" that have been borrowed by any user but not yet returned.
books = mycol.find({
    "author": "Author 5",
    "borrowed_by.return_date": None
})

for book in books:
    print(book)

{'_id': ObjectId('667682f5a42aa2dba072345a'), 'title': 'Book Title 4', 'author': 'Author 5', 'genre': 'Dystopian', 'published_year': 1954, 'copies_available': 91, 'borrowed_by': [{'user_id': ObjectId('667682f5a42aa2dba0723459'), 'borrowed_date': datetime.datetime(2024, 9, 30, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f5a42aa2dba072346e'), 'title': 'Book Title 14', 'author': 'Author 5', 'genre': 'Fantasy', 'published_year': 1964, 'copies_available': 81, 'borrowed_by': [{'user_id': ObjectId('667682f5a42aa2dba072346d'), 'borrowed_date': datetime.datetime(2024, 12, 19, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f6a42aa2dba0723482'), 'title': 'Book Title 24', 'author': 'Author 5', 'genre': 'Adventure', 'published_year': 1974, 'copies_available': 41, 'borrowed_by': [{'user_id': ObjectId('667682f6a42aa2dba0723481'), 'borrowed_date': datetime.datetime(2024, 10, 30, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f6a42aa2dba0723496'), 'title': 'Book Title 34', 'auth

In [4]:
#List all books published before the year 1980 and have more than 5 copies available.
books = mycol.find({
    "published_year": { "$lt": 1980 },
    "copies_available": { "$gt": 5 }
})

for book in books:
    print(book)

{'_id': ObjectId('667682f5a42aa2dba0723452'), 'title': 'Book Title 0', 'author': 'Author 1', 'genre': 'Science Fiction', 'published_year': 1950, 'copies_available': 71, 'borrowed_by': [{'user_id': ObjectId('667682f5a42aa2dba0723451'), 'borrowed_date': datetime.datetime(2024, 10, 9, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f5a42aa2dba0723454'), 'title': 'Book Title 1', 'author': 'Author 2', 'genre': 'Adventure', 'published_year': 1951, 'copies_available': 41, 'borrowed_by': [{'user_id': ObjectId('667682f5a42aa2dba0723453'), 'borrowed_date': datetime.datetime(2024, 3, 15, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f5a42aa2dba0723456'), 'title': 'Book Title 2', 'author': 'Author 3', 'genre': 'Fiction', 'published_year': 1952, 'copies_available': 51, 'borrowed_by': [{'user_id': ObjectId('667682f5a42aa2dba0723455'), 'borrowed_date': datetime.datetime(2024, 11, 24, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f5a42aa2dba0723458'), 'title': 'Book Title 3', 'au

In [5]:
#Find the top 5 most recently published books in the "Fantasy" genre.
books = mycol.find({
    "genre": "Fantasy"
}).sort("published_year", pymongo.DESCENDING).limit(5)

for book in books:
    print(book)

{'_id': ObjectId('6676831ba42aa2dba0723594'), 'title': 'Book Title 68', 'author': 'Author 9', 'genre': 'Fantasy', 'published_year': 2018, 'copies_available': 71, 'borrowed_by': [{'user_id': ObjectId('6676831ba42aa2dba0723593'), 'borrowed_date': datetime.datetime(2024, 5, 6, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f6a42aa2dba07234d2'), 'title': 'Book Title 64', 'author': 'Author 5', 'genre': 'Fantasy', 'published_year': 2014, 'copies_available': 51, 'borrowed_by': [{'user_id': ObjectId('667682f6a42aa2dba07234d1'), 'borrowed_date': datetime.datetime(2024, 1, 27, 0, 0), 'return_date': None}]}
{'_id': ObjectId('6676831ba42aa2dba0723588'), 'title': 'Book Title 62', 'author': 'Author 3', 'genre': 'Fantasy', 'published_year': 2012, 'copies_available': 91, 'borrowed_by': [{'user_id': ObjectId('6676831ba42aa2dba0723587'), 'borrowed_date': datetime.datetime(2024, 5, 19, 0, 0), 'return_date': None}]}
{'_id': ObjectId('667682f6a42aa2dba07234cc'), 'title': 'Book Title 61', 'author': '

In [6]:
#Count the number of books available for each genre.
book_genre = mycol.aggregate([
    {
        "$group": {
            "_id": "$genre",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
])

for result in book_genre:
    print(f"Genre: {result['_id']}, Count: {result['count']}")

Genre: Fantasy, Count: 67
Genre: Dystopian, Count: 60
Genre: Science Fiction, Count: 59
Genre: Adventure, Count: 57
Genre: Fiction, Count: 49


In [10]:
# Find all books that have never been borrowed by any user.
books = mycol.find({
    "borrowed_by": {"$exists": False}
})

for book in books:
    print(book)